## imports

In [1]:
import ast
import inspect
import types
import numpy as np
import numba as nb
import pandas as pd
import time
import logging
import functools
import copy
import sys
from typing import Callable, Type, Dict, Tuple, Any
from numpy.lib.stride_tricks import sliding_window_view
logger = logging.getLogger()
logger.setLevel(0)

## dev imports

In [2]:
import astpretty

## Current state

In [3]:


def create_callmap_function_ast(mapping: Dict[str, int]) -> ast.FunctionDef:
    # Create the body of the callmap function
    body = []
    for key, value in mapping.items():
        compare = ast.Compare(
            left=ast.Name(id='x', ctx=ast.Load()),
            ops=[ast.Eq()],
            comparators=[ast.Constant(value=key)]
        )
        body.append(
            ast.If(
                test=compare,
                body=[ast.Return(value=ast.Constant(value=value))],
                orelse=[]
            )
        )
    
    # Add a default return statement
    body.append(ast.Return(value=ast.Name(id='x', ctx=ast.Load())))

    # Create the function definition
    func_def = ast.FunctionDef(
        name='callmap',
        args=ast.arguments(
            posonlyargs=[],
            args=[ast.arg(arg='x')],
            vararg=None,
            kwonlyargs=[],
            kw_defaults=[],
            kwarg=None,
            defaults=[]
        ),
        body=body,
        decorator_list=[],
        returns=None
    )
    return func_def

class SubscriptReplacer(ast.NodeTransformer):
    def __init__(self, arg_name):
        self.arg_name = arg_name

    def visit_Subscript(self, node):
        if isinstance(node.value, ast.Name) and node.value.id == self.arg_name:
            # Check for Python version compatibility
            if sys.version_info >= (3, 9):
                # Python 3.9 and later
                old_slice = node.slice
            else:
                # Python 3.8 and earlier
                old_slice = node.slice.value if isinstance(node.slice, ast.Index) else node.slice

            # Wrap the subscript in a call to callmap
            node.slice = ast.Call(
                func=ast.Name(id='callmap', ctx=ast.Load()),
                args=[old_slice],
                keywords=[]
            )
        return self.generic_visit(node) 

def create_transformed_function_ast(original_func: Callable, mapping: Dict[str, int]) -> Tuple[ast.AST, ast.AST, ast.AST]:
    # Parse the original function
    original_tree = ast.parse(inspect.getsource(original_func))
    arg_name = original_tree.body[0].args.args[0].arg
    
    # Rename the original function
    original_tree.body[0].name = 'temporary'
    
    # Apply the AST transformation
    replacer = SubscriptReplacer(arg_name)
    original_tree = replacer.visit(original_tree)
    ast.fix_missing_locations(original_tree)

    # Replace dictionary accesses with callmap in the original function
    # This would be similar to the code in SubscriptReplacer

    # Create a new function that applies 'temporary' over an array
    loop_base_func_str = f"""
def {original_func.__qualname__}_loop(Z):
    n = Z.shape[0]
    res = np.zeros((n, 1))
    for i in nb.prange(n):
        res[i, 0] = temporary(Z[i, :])
    return res
    """
    vectorized_base_func_str = f"""
def {original_func.__qualname__}_vectorized(Z):
    return temporary(Z.T)
    """
    loop_func_tree = ast.parse(loop_base_func_str)
    vectorize_func_tree = ast.parse(vectorized_base_func_str)

    return original_tree, loop_func_tree, vectorize_func_tree

def numba_decorate(func_tree: ast.AST, nopython: bool = True, nogil: bool = True, parallel: bool = True) -> ast.AST:
    # # Add Numba JIT decorator
    nb_compyled_func_tree = copy.deepcopy(ast.fix_missing_locations(func_tree))
    numba_decorator = ast.Call(
        func=ast.Attribute(value=ast.Name(id='nb', ctx=ast.Load()), attr='jit', ctx=ast.Load()),
        args=[],
        keywords=[
            ast.keyword(arg='nopython', value=ast.Constant(value=nopython)),
            ast.keyword(arg='nogil', value=ast.Constant(value=nogil)),
            ast.keyword(arg='parallel', value=ast.Constant(value=parallel))
        ]
    )
    nb_compyled_func_tree.body[0].decorator_list.append(numba_decorator)
    nb_compyled_func_tree.body[0].name += '_nb_compyled'
    return ast.fix_missing_locations(nb_compyled_func_tree)

def encapulate(wrap_tree: ast.AST, callmap_tree: ast.AST, original_tree: ast.AST) -> ast.AST:
    wrap_tree.body[0].body.insert(0, callmap_tree.body[0])
    wrap_tree.body[0].body.insert(1, original_tree.body[0])
    return ast.fix_missing_locations(wrap_tree)

def compile_tree(built_func_tree: ast.AST, exec_globals: Dict[str, Any], qualname: str, build_qualifier: str) -> Dict:
    try:
        exec(compile(built_func_tree, filename="<ast>", mode="exec"), exec_globals)
        return {build_qualifier: exec_globals[qualname + build_qualifier]}
    except Exception as e:
        logger.warning(e)
    return {}

def _prepare_funcs(original_func: ast.AST, mapping: Dict[str, int]) -> Dict[str, Callable]:
    exec_globals = globals().copy()
    exec_globals.update({'np': np, 'nb': nb})
    callmap_func_ast = create_callmap_function_ast(mapping)
    callmap_func_tree = ast.fix_missing_locations(ast.Module(body=[callmap_func_ast], type_ignores=[]))
    original_tree, loop_func_tree, vectorize_func_tree = create_transformed_function_ast(original_func, mapping)

    loop_func_tree = encapulate(loop_func_tree, callmap_func_tree, original_tree)
    vectorize_func_tree = encapulate(vectorize_func_tree, callmap_func_tree, original_tree)
    
    available_funcs = {}
    available_funcs.update(compile_tree(vectorize_func_tree, exec_globals, original_func.__qualname__, '_vectorized'))
    available_funcs.update(compile_tree(loop_func_tree, exec_globals, original_func.__qualname__, '_loop'))
    
    nb_compyled_loop_func_tree = numba_decorate(loop_func_tree)
    nb_compyled_vectorize_func_tree = numba_decorate(vectorize_func_tree)

    available_funcs.update(compile_tree(nb_compyled_vectorize_func_tree, exec_globals, original_func.__qualname__, '_vectorized_nb_compyled'))
    available_funcs.update(compile_tree(nb_compyled_loop_func_tree, exec_globals, original_func.__qualname__, '_loop_nb_compyled'))

    return available_funcs

def make_class_decorator(function_decorator: Callable) -> Callable:
    """
    Creates a class decorator from a given function decorator.

    Args:
        function_decorator (Callable): A function decorator to be applied to class methods.

    Returns:
        Callable: A class decorator.
    """
    @functools.wraps(function_decorator)
    def class_decorator(cls: Type) -> Type:
        """
        The class decorator generated from the function decorator.

        Args:
            cls (Type): The class to which the decorator is applied.

        Returns:
            Type: The decorated class.
        """
        for attr_name, attr_value in cls.__bases__[0].__dict__.items():
            if callable(attr_value) and not attr_name.startswith('_') and attr_name not in cls.__dict__:
                setattr(cls, attr_name, function_decorator(attr_value))
        for attr_name, attr_value in cls.__dict__.items():
             if callable(attr_value) and not attr_name.startswith('_'):
                setattr(cls, attr_name, function_decorator(attr_value))
        return cls
    return class_decorator

def autowrap_pandas_return(fn: Callable) -> Callable:
    """
    Decorator to add validation and error handling to class methods.

    Args:
        fn (Callable): The original method of the class.

    Returns:
        Callable: The decorated method with added validation and error handling.
    """
    @functools.wraps(fn)
    def wrapper(self, *args, **kwargs):
        if self._outside_call:
            self._outside_call = False
            res = fn(self, *args, **kwargs)
            if isinstance(res, pd.DataFrame):
                res = pandopt(res)
            self._outside_call = True
            return res
        return fn(self, *args, **kwargs)
    return wrapper

@make_class_decorator(autowrap_pandas_return)
class pandopt(pd.DataFrame):
    _compiled_func = None
    _outside_call = True

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._compiled_func = {}

    @property
    def __name__(self):
        return functools.reduce(lambda x, y: x + y, self.name_to_index.keys())

    @property
    def colname_to_colnum(self):
        return {k: i for i, k in enumerate(self.columns)}

    @property
    def rowname_to_rownum(self):
        return {k: i for i, k in enumerate(self.index)}
    
    def _compiled_qualifier(self, func_qualifier, mapper):
        return hash(functools.reduce(lambda x, y: f'{x}&{y}', mapper) + func_qualifier)

    def apply(self, func, axis = 0, *args, pandas_fallback = False, **kwargs):
        if pandas_fallback: 
            logger.warning(f'{__class__} finish in pandas fallback for func {func}')
            return super().apply(func, axis = 0, *args, **kwargs)
        if args or kwargs:
            logger.warning(f'{__class__} apply only supports func and axis arguments, using default pandas apply')
            return super().apply(func, axis = 0, *args, **kwargs)
        return pandopt((self._compiled_func.get((name:=self._compiled_qualifier(func_qualifier = func.__qualname__, mapper=(mapper:=self.colname_to_colnum if axis else self.rowname_to_rownum)))) or self._build_apply_versions(func, mapper, name))(self.to_numpy() if axis else self.to_numpy().T), index = self.index if axis else self.columns)

    def _with_fallback_wrap(self, apply_func_dict):
        def _with_protects(*args, **kwargs):
            for key in ('_vectorized_nb_compyled', '_loop_compyled', '_vectorized', '_loop'):
                if key not in apply_func_dict:
                    continue
                try:
                    return apply_func_dict[key](*args, **kwargs)
                except:
                    apply_func_dict.pop(key)
            return self.apply(*args, pandas_fallback = True, **kwargs)
        return _with_protects
    
    def _build_apply_versions(self, func, map, name):
        self._compiled_func[name] = self._with_fallback_wrap(_prepare_funcs(func, map))
        return self._compiled_func[name]

    


In [ ]:
# @nb.jit(fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', no_cfunc_wrapper=True, no_rewrites=True,nopython=True, nogil=True, parallel = True)
def mean(x):
    res=x.shape[1]
    for k in nb.prange(x.shape[1]):
        res[k]= np.mean(x[:,k])
    return res

def mmean(x):
    return functools.reduce(lambda k, z: k + z, x)/len(x)

df=pd.DataFrame(data=np.random.randn(10000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)
# %timeit dfx.mean()
# %timeit df.mean()
# %timeit np.mean(dfx.to_numpy(), axis=0)
# %timeit mean(dfx.to_numpy())
# %timeit dfx.apply(mmean)
# %timeit dfx.apply(mean)


In [19]:
np.mean(dfx.to_numpy(), axis=0),  mean(dfx.to_numpy()), df.mean()

(array([ 0.00293472, -0.00066693,  0.00986487,  0.00569817], dtype=float32),
 0.004457703104591451,
 A    0.002935
 B   -0.000667
 C    0.009865
 D    0.005698
 dtype: float32)

In [10]:
%timeit np.mean(dfx.to_numpy())
%timeit mean(dfx.to_numpy())

13.7 µs ± 33.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
23.4 µs ± 5.73 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
@nb.jit(fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', no_cfunc_wrapper=True,nopython=True, nogil=True, parallel = True)
def var1(x):
    K=x.shape
    print(x)
    res = np.zeros((K,1))
    for k in nb.prange():
        res[k,0] = np.var(x[:,k])
    return res

@nb.jit(fastmath=True, forceinline=True, looplift=True, inline='always', target_backend='host', no_cfunc_wrapper=True, no_rewrites=True,nopython=True, nogil=True, parallel = True)
def var2(x):
    m = np.sum(x, axis=0)/x.shape[0]
    return (m**2 - m)
    
var2(df.to_numpy())

In [ ]:
a=0
def test(x):
    global a
    a+=1
    return np.mean(x)

test(df.rolling(4))

In [5]:
def f(x):
    return np.mean(x)

def f2(x):
    return functools.reduce(lambda k, z: k + z, x)/len(x)

%timeit df.mean(axis=1)
%timeit df.apply(np.mean, axis=1)
%timeit dfx.apply(f, axis=1)


1.14 ms ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
244 ms ± 6.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


/tmp/ipykernel_535606/1627591937.py:8: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  comparators=[ast.Constant(s=key)]
/tmp/ipykernel_535606/1627591937.py:13: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  body=[ast.Return(value=ast.Constant(n=value))],


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/remi/.pyenv/versions/3.12.1/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_535606/4046254626.py", line 9, in <module>
    get_ipython().run_line_magic('timeit', 'dfx.apply(f, axis=1)')
  File "/home/remi/.pyenv/versions/3.12.1/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2456, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "/home/remi/.pyenv/versions/3.12.1/lib/python3.12/site-packages/IPython/core/magics/execution.py", line 1185, in timeit
    time_number = timer.timeit(number)
                  ^^^^^^^^^^^^^^^^^^^^
  File "/home/remi/.pyenv/versions/3.12.1/lib/python3.12/site-packages/IPython/core/magics/execution.py", line 173, in timeit
    timing = self.inner(it, self.timer)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<magic-timeit>", line 1, in i

In [6]:
def mymean(x):
    return functools.reduce(lambda k, z: k + z, x)/len(x)

def somemean(x):
    return sum(x)/len(x)

df=pd.DataFrame(data=np.random.randn(1000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)


%time df.mean(axis=1)
%time df.apply(np.mean, axis=1)
%time dfx.apply(mymean, axis=1)
%time dfx.apply(somemean, axis=1)

print(df.mean(axis=1))
print(df.apply(np.mean, axis=1))
print(dfx.apply(mymean, axis=1))
print(dfx.apply(somemean, axis=1))


CPU times: user 87.2 ms, sys: 3.81 ms, total: 91.1 ms
Wall time: 91.1 ms
CPU times: user 21.7 s, sys: 39.2 ms, total: 21.7 s
Wall time: 21.7 s
CPU times: user 17.1 ms, sys: 200 µs, total: 17.3 ms
Wall time: 22.9 ms
CPU times: user 56.5 ms, sys: 88 µs, total: 56.6 ms
Wall time: 57.1 ms
0        -0.261240
1        -0.154031
2         0.979302
3         0.651981
4         0.472666
            ...   
999995   -0.464518
999996    0.339989
999997    0.715249
999998   -0.618368
999999    0.790618
Length: 1000000, dtype: float32


/tmp/ipykernel_535606/1627591937.py:8: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  comparators=[ast.Constant(s=key)]
/tmp/ipykernel_535606/1627591937.py:13: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  body=[ast.Return(value=ast.Constant(n=value))],
/tmp/ipykernel_535606/1627591937.py:8: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  comparators=[ast.Constant(s=key)]
/tmp/ipykernel_535606/1627591937.py:13: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  body=[ast.Return(value=ast.Constant(n=value))],


0        -0.261240
1        -0.154031
2         0.979302
3         0.651981
4         0.472666
            ...   
999995   -0.464518
999996    0.339989
999997    0.715249
999998   -0.618368
999999    0.790618
Length: 1000000, dtype: float32
               0
0      -0.261240
1      -0.154031
2       0.979302
3       0.651981
4       0.472666
...          ...
999995 -0.464518
999996  0.339989
999997  0.715249
999998 -0.618368
999999  0.790618

[1000000 rows x 1 columns]
               0
0      -0.261240
1      -0.154031
2       0.979302
3       0.651981
4       0.472666
...          ...
999995 -0.464518
999996  0.339989
999997  0.715249
999998 -0.618368
999999  0.790618

[1000000 rows x 1 columns]


In [120]:
[attr for attr in dir(df) if hasattr(np, attr) and "__" not in attr]

/tmp/ipykernel_548805/3868163403.py:1: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  [attr for attr in dir(df) if hasattr(np, attr) and "__" not in attr]


['abs',
 'add',
 'all',
 'any',
 'clip',
 'copy',
 'cov',
 'cumprod',
 'cumsum',
 'diff',
 'divide',
 'dot',
 'dtypes',
 'empty',
 'info',
 'insert',
 'isin',
 'max',
 'mean',
 'median',
 'min',
 'mod',
 'multiply',
 'ndim',
 'pad',
 'prod',
 'product',
 'quantile',
 'round',
 'shape',
 'size',
 'squeeze',
 'stack',
 'std',
 'subtract',
 'sum',
 'swapaxes',
 'take',
 'transpose',
 'var',
 'where']

CPU times: user 1.41 ms, sys: 88 µs, total: 1.5 ms
Wall time: 2.15 ms


,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,-0.754639,0.515742,-0.274488,-0.104898
3,1.498076,0.201619,-0.274488,-0.079058
4,0.153899,1.713065,0.022485,-0.079058
...,...,...,...,...
95,-0.183078,-0.022846,1.334153,0.989433
96,-0.183078,-1.126700,0.778397,-0.270090
97,-0.842353,-1.126700,0.774261,-0.771215
98,0.350380,-0.296916,-0.224929,-0.771215


In [128]:
pd.DataFrame(np.mean(sliding_window_view(df.to_numpy(), 5, axis=0), axis=2)).values - 1).abs().max()

ValueError: Since axis is `None`, must provide window_shape for all dimensions of `x`; got 1 window_shape elements and `x.ndim` is 2.

In [ ]:

@nb.jit(nopython=True, nogil=True, parallel=True)
def cdmtest_func(Z):
    def callmap(x):
        if x == 'A':
            return 0
        elif x == 'B':
            return 1
        elif x == 'C':
            return 2
        elif x == 'D':
            return 3
        return x
    def tmporary(x):
        return np.kurt(x)
    n = Z.shape[0]
    res = np.zeros((n, 1))
    for i in nb.prange(5, n):
        res[i,0] = np.var(Z[i-5:i,:])
    return res


def mymean(x):
    return functools.reduce(lambda k, z: k + z, x)/len(x)


df=pd.DataFrame(data=np.random.randn(10000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)

%time cdmtest_func(df.to_numpy())

%time df.rolling(5).var()
%time df.rolling(5).apply(np.var)
%time dfx.rolling(5).apply(mymean)

In [ ]:

def test_func(z):
    x = (z['A']+z['B'])
    x = z['B']*z['D']
    return x / z['B']

@nb.jit(nopython=True, nogil=True, parallel=True, cache=True)
def test_funcv(z):
    x = (z[:,0]+z[:,1])
    x = z[:,0]*z[:,3]
    return x / z[:,1]

@nb.jit(nopython=True, nogil=True, cache=True)
def test_funcd(z):
    x = (z[0]+z[1])
    x = z[0]*z[3]
    return x / z[1]

@nb.jit(nopython=True, nogil=True, parallel=True, cache=True)
def test_funcdd(z):
    for i in nb.prange(len(z)):
        z[i,0]=test_funcd(z[i])
    return z[:,0]
    
def rowtest_func(z):
    x = (z['A']+z['B'])
    if x > 0:
        return z['B']*z['D']
    return x / z['B']

@nb.jit(nopython=True, nogil=True, cache=True)
def rowtest_funcdi(z):
    x = (z[0]+z[1])
    if x > 0:
        return z[0]*z[3]
    return x / z[1]

@nb.jit(nopython=True, nogil=True, parallel=True)
def rowtest_funcddo(z):
    n = len(z)
    result = np.zeros((n,1), dtype=np.float32)
    for i in nb.prange(n):
        result[i,0] = rowtest_funcdi(z[i,:])
    return result

@nb.jit(nopython=True, nogil=True, parallel=True)
def rowtest_funcdd(z):
    n = len(z)
    result = np.zeros((n,1), dtype=np.float32)
    for i in nb.prange(n):
        x = (z[i, 0]+z[i, 1])
        if x > 0:
            result[i,0] = z[i, 0]*z[i, 3]
            continue
        result[i,0] = z[i,0]
        continue
    return result



@nb.jit(nopython=True, nogil=True, parallel=True)
def rowtest_funcdv(z):
    x = (z[:,0]+z[:,1])
    return ((x > 0) * z[:,0]*z[:,3]) + ((x < 0) * x / z[:,1])


df=pd.DataFrame(data=np.random.randn(10**3,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
arr=df.astype(np.float32).to_numpy()
print(df.apply(rowtest_func, axis=1))
print(rowtest_funcdd(arr.copy()), rowtest_funcdd(arr.copy()).shape, np.sum(rowtest_funcdd(arr.copy())))
print(rowtest_funcdv(arr.copy()), rowtest_funcdv(arr.copy()).shape, np.sum(rowtest_funcdv(arr.copy())))

print(df.apply(test_func, axis=1))
print(test_funcdd(arr.copy()), test_funcdd(arr.copy()).shape, np.sum(test_funcdd(arr.copy())))
print(test_funcv(arr.copy()), test_funcv(arr.copy()).shape, np.sum(test_funcv(arr.copy())))

df.rolling(4)

In [ ]:
%timeit var1(df.to_numpy())
%timeit var2(df.to_numpy())
%timeit np.var(df.to_numpy(), axis=0)
%timeit df.var()

## Func compilation / live modification performance test

In [4]:
# Example usage
def simple_start(z):
    x = (z['A'] + z['B']) / z['C']
    x += z['B'] * z['D']
    return x / z['B']

def harder_func(z):
    x = (z['A'] + z['B']) / z['C']
    if x > 0:
        return x / z['B']
    x += z['B'] * z['D']
    return x * z['B']

def harder2_func(z):
    x = (z['A'] + z['B']) / z['C']
    if (k:=z['A']-z['C']) > (j:=z['B']/z['D']):
        return x / k
    x *= j
    return x - k if k > z['C'] else x + k

def harder3_func(z):
    g=lambda a, b: a if abs(a) > abs(b) else - 2 * (b**(-a))
    x = (z['A'] + z['B']) / z['C']
    if (k:=z['A']-z['C']) > (j:=g(z['B'],z['D'])):
        return j / k
    x *= j
    return x - k if k > z['C'] else x + k


In [ ]:

df=pd.DataFrame(data=np.random.randn(100000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
mapping = {k: i for i, k in enumerate(df.columns)}

for func_test in [simple_start, harder_func, harder2_func, harder3_func]:
    print('Testing ', func_test.__name__)
    prepared_funcs = _prepare_funcs(func_test, mapping)
    try:
        print('df.apply(f, axis=1):', end=' ')
        %timeit df.apply(func_test, axis=1)
        print('checksum: ', np.sum(df.apply(func_test, axis=1)))
    except Exception as e:
        print('df.apply(f, axis=1) FAILED : ', e)
    try:
        print('test_func(df):', end=' ')
        %timeit func_test(df)
        print('checksum: ', np.sum(func_test(df)))
    except Exception as e:
        print('test_func(df) FAILED : ', e)
    try:
        print('_loop(df):', end=' ')
        %timeit prepared_funcs['_loop'](df.to_numpy())
        print('checksum: ', np.sum(prepared_funcs['_loop'](df.to_numpy())))
    except Exception as e:
        print('safe_func(df) FAILED : ', e)
    try:
        print('_vectorize(df):', end=' ')
        %timeit prepared_funcs['_vectorize'](df.to_numpy())
        print('checksum: ', np.sum(prepared_funcs['_vectorize'](df.to_numpy().T)))
    except Exception as e:
        print('vectorized_func(df) FAILED : ', e)
    try:
        print('_loop_nb_compyled(df):', end=' ')
        %timeit prepared_funcs['_loop_nb_compyled'](df.to_numpy())
        print('checksum: ', np.sum(prepared_funcs['_loop_nb_compyled'](df.to_numpy())))
    except Exception as e:
        print('opt_func(df) FAILED : ', e)
    try:
        print('_vectorized_nb_compyled(df):', end=' ')
        %timeit prepared_funcs['_vectorized_nb_compyled'](df.to_numpy())
        print('checksum: ', np.sum(prepared_funcs['_vectorized_nb_compyled'](df.to_numpy().T)))
    except Exception as e:
        print('opt_func(df) FAILED : ', e)
    print('\n\n\n')

In [92]:
import numpy as np
from numpy.lib.stride_tricks import as_strided, sliding_window_view
def aid(x):
    # This function returns the memory
    # block address of an array.
    return x.__array_interface__['data'][0]

n = 5
k = 2
a = np.random.randn(10,3)
ax = aid(a)

aid(a), aid(as_strided(a, (k, n), (8, 8)))


(93843876078064, 93843876078064)

In [41]:
df=pd.DataFrame(data=np.random.randn(100,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)

In [88]:
np.mean(roll(df.to_numpy(),5), axis=2).shape

(100, 4)

,A,B,C,D
0,2.655950e-01,2.216493e-01,1.200793e-02,7.001556e-01
1,-9.807208e-01,9.708073e-02,3.352248e-01,1.042880e+00
2,3.349523e-01,-8.364432e-02,-7.467062e-01,-8.917299e-02
3,3.136172e-01,-1.546154e-01,-1.230396e+00,2.513432e-01
4,-1.757043e-01,5.161352e-01,-4.074371e-01,7.426448e-01
...,...,...,...,...
95,5.386472e+22,8.006274e+00,1.194297e-07,1.267386e-10
96,3.026565e+24,9.451758e-42,NaN,NaN
97,3.779411e+36,9.481500e+29,5.580278e+30,1.428786e+31
98,6.061064e+34,1.501139e+28,1.881535e+31,1.454168e+31


In [106]:
df.rolling(5).mean()

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,0.337746,0.762621,-0.168195,0.267237
...,...,...,...,...
95,0.162133,-0.010538,1.046065,0.299708
96,0.268621,-0.318093,1.027814,0.044403
97,0.023694,-0.432432,0.806661,-0.114526
98,-0.222032,-0.188639,0.494844,-0.538666


In [115]:
(df.rolling(5).mean().dropna() / 

A    2.296739e-05
B    6.602486e-07
C    1.979068e-06
D    3.124961e-06
dtype: float64

In [81]:
df.rolling(5).mean()

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,0.337746,0.762621,-0.168195,0.267237
...,...,...,...,...
95,0.162133,-0.010538,1.046065,0.299708
96,0.268621,-0.318093,1.027814,0.044403
97,0.023694,-0.432432,0.806661,-0.114526
98,-0.222032,-0.188639,0.494844,-0.538666


In [82]:
pd.DataFrame(np.mean(roll(df.to_numpy(),5), axis=2))

,0,1,2,3
0,0.265595,0.456847,0.298801,0.386469
1,0.456847,0.298801,0.386469,0.178652
2,0.298801,0.386469,0.178652,0.221649
3,0.386469,0.178652,0.221649,-0.066853
4,0.178652,0.221649,-0.066853,-0.091192
...,...,...,...,...
95,0.742645,0.812533,0.472504,0.308140
96,0.812533,0.472504,0.308140,0.302930
97,0.472504,0.308140,0.302930,0.238306
98,0.308140,0.302930,0.238306,-0.176305


In [55]:
roll(df.to_numpy(),5)[:,0,:]

array([[-0.7546394 ,  0.51574165,  0.17369436, -0.10489792,  1.4980764 ],
       [ 0.51574165,  0.17369436, -0.10489792,  1.4980764 ,  0.20161924],
       [ 0.17369436, -0.10489792,  1.4980764 ,  0.20161924, -0.27448794],
       [-0.10489792,  1.4980764 ,  0.20161924, -0.27448794,  0.6120349 ],
       [ 1.4980764 ,  0.20161924, -0.27448794,  0.6120349 , -1.1439844 ],
       [ 0.20161924, -0.27448794,  0.6120349 , -1.1439844 ,  1.7130646 ],
       [-0.27448794,  0.6120349 , -1.1439844 ,  1.7130646 , -1.2408917 ],
       [ 0.6120349 , -1.1439844 ,  1.7130646 , -1.2408917 , -0.3961847 ],
       [-1.1439844 ,  1.7130646 , -1.2408917 , -0.3961847 ,  1.9353764 ],
       [ 1.7130646 , -1.2408917 , -0.3961847 ,  1.9353764 , -0.71648335],
       [-1.2408917 , -0.3961847 ,  1.9353764 , -0.71648335,  0.47822297],
       [-0.3961847 ,  1.9353764 , -0.71648335,  0.47822297, -0.07905765],
       [ 1.9353764 , -0.71648335,  0.47822297, -0.07905765,  0.15389891],
       [-0.71648335,  0.47822297, -0.0

In [52]:
df.rolling(5).mean()

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,0.337746,0.762621,-0.168195,0.267237
...,...,...,...,...
95,0.162133,-0.010538,1.046065,0.299708
96,0.268621,-0.318093,1.027814,0.044403
97,0.023694,-0.432432,0.806661,-0.114526
98,-0.222032,-0.188639,0.494844,-0.538666


In [26]:
as_strided(a, (k, n - k + 1),
                      (x.itemsize, x.itemsize)

array([[ 0.27001452, -1.11780581, -0.23136538],
       [-0.62645256,  0.32645478,  0.6114013 ],
       [ 0.73894834,  0.15185684,  1.57561642],
       [-0.09820861,  0.31168666, -1.11952458],
       [-0.12519364,  0.8484354 , -0.04201383],
       [ 1.50032206, -1.4191039 , -0.91795048],
       [-0.03379298, -0.17146464, -1.44574201],
       [-0.09940819, -0.23099795,  0.30573739],
       [-0.16094288, -0.63948968,  0.16872367],
       [-0.44835185, -0.16928613, -0.07846052]])

In [23]:

as_strided(a, a.shape)

array([[ 0.09853989,  0.85480222,  0.96302139],
       [ 0.01785596,  3.19971691,  0.34999999],
       [ 1.25770184, -0.57590728,  0.24321342],
       [-0.13461559,  1.54272491, -0.28146535],
       [ 0.79177697,  0.15738101, -0.46336858],
       [-1.05338002,  1.37710085, -0.95953717],
       [-1.41492694,  1.90493116, -0.37914363],
       [-1.00195352, -0.39177092,  0.8631298 ],
       [-0.32857131, -0.6975006 , -0.82957972],
       [-0.89631085, -1.04454778, -1.28107639]])

In [5]:
df=pd.DataFrame(data=np.random.randn(1000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)
dfx.apply(simple_start, axis=1)

for func_test in [simple_start, harder_func, harder2_func, harder3_func]:
    print('Testing ', func_test.__name__)

    default = df.apply(func_test, axis=1)
    new = dfx.apply(func_test, axis=1)

    if sum(default - new)

    %timeit df.apply(func_test, axis=1)

    %timeit dfx.apply(func_test, axis=1)
    

Testing  simple_start
9.39 s ± 34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
598 µs ± 8.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Testing  harder_func
9.93 s ± 51.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.38 s ± 30.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Testing  harder2_func
11 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.42 s ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Testing  harder3_func


/tmp/ipykernel_548805/1400378024.py:22: RuntimeWarning: invalid value encountered in scalar power
  g=lambda a, b: a if abs(a) > abs(b) else - 2 * (b**(-a))


13 s ± 53.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<ast>:2: RuntimeWarning: invalid value encountered in scalar power


2.7 s ± 6.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
def simple_start(z):
    x = (z['A'] + z['B']) / z['C']
    x += z['B'] * z['D']
    return x / z['B']


df=pd.DataFrame(data=np.random.randn(1000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)

%time df.apply(simple_start, axis=1)

%time dfx.apply(simple_start, axis=1)

CPU times: user 9.15 s, sys: 67.6 ms, total: 9.21 s
Wall time: 9.22 s
CPU times: user 1.11 s, sys: 3.11 ms, total: 1.11 s
Wall time: 1.08 s


,0
0,-3.959772
1,-0.907447
2,1.253053
3,-2.062477
4,3.794123
...,...
999995,32.175377
999996,-1.739293
999997,0.288636
999998,-4.231944


In [11]:
def simple_start(z):
    x = (z['A'] + z['B']) / z['C']
    x += z['B'] * z['D']
    return x / z['B']

df=pd.DataFrame(data=np.random.randn(10000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)

%time df.apply(simple_start, axis=1)

%time dfx.apply(simple_start, axis=1)

CPU times: user 1min 30s, sys: 809 ms, total: 1min 31s
Wall time: 1min 31s
CPU times: user 1.4 s, sys: 285 ms, total: 1.69 s
Wall time: 1.08 s


,0
0,-6.192018
1,-4.756915
2,3.930151
3,7.628836
4,2.251130
...,...
9999995,2.934612
9999996,0.472286
9999997,38.893909
9999998,3.185539


In [141]:
df=pd.DataFrame(data=np.random.randn(1000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)

%time df.rolling(10).var()
%time pd.DataFrame(np.var(sliding_window_view(df.to_numpy(), 10, axis=0), axis=2), columns=df.columns, index=df.index[9:])
#%time dfx.apply(simple_start, axis=1)

CPU times: user 52.9 ms, sys: 7.82 ms, total: 60.8 ms
Wall time: 60.8 ms
CPU times: user 408 ms, sys: 12 ms, total: 420 ms
Wall time: 420 ms


,A,B,C,D
9,1.037629,0.601060,0.774779,0.981819
10,0.822783,1.190935,0.957456,1.004601
11,0.809825,1.529604,1.105945,1.015404
12,1.363136,1.404535,1.059775,1.045752
13,0.991759,1.400987,0.899653,0.837341
...,...,...,...,...
999995,0.429226,0.397719,1.137106,1.219176
999996,0.707291,0.261690,0.917446,1.251031
999997,0.742810,0.358475,0.941837,1.144907
999998,0.632274,0.538827,0.967900,0.856543


In [ ]:


def create_callmap_function_ast(mapping: Dict[str, int]) -> ast.FunctionDef:
    # Create the body of the callmap function
    body = []
    for key, value in mapping.items():
        compare = ast.Compare(
            left=ast.Name(id='x', ctx=ast.Load()),
            ops=[ast.Eq()],
            comparators=[ast.Constant(value=key)]
        )
        body.append(
            ast.If(
                test=compare,
                body=[ast.Return(value=ast.Constant(value=value))],
                orelse=[]
            )
        )
    
    # Add a default return statement
    body.append(ast.Return(value=ast.Name(id='x', ctx=ast.Load())))

    # Create the function definition
    func_def = ast.FunctionDef(
        name='callmap',
        args=ast.arguments(
            posonlyargs=[],
            args=[ast.arg(arg='x')],
            vararg=None,
            kwonlyargs=[],
            kw_defaults=[],
            kwarg=None,
            defaults=[]
        ),
        body=body,
        decorator_list=[],
        returns=None
    )
    return func_def

class SubscriptReplacer(ast.NodeTransformer):
    def __init__(self, arg_name):
        self.arg_name = arg_name

    def visit_Subscript(self, node):
        if isinstance(node.value, ast.Name) and node.value.id == self.arg_name:
            # Check for Python version compatibility
            if sys.version_info >= (3, 9):
                # Python 3.9 and later
                old_slice = node.slice
            else:
                # Python 3.8 and earlier
                old_slice = node.slice.value if isinstance(node.slice, ast.Index) else node.slice

            # Wrap the subscript in a call to callmap
            node.slice = ast.Call(
                func=ast.Name(id='callmap', ctx=ast.Load()),
                args=[old_slice],
                keywords=[]
            )
        return self.generic_visit(node) 

def create_transformed_function_ast(original_func: Callable, mapping: Dict[str, int]) -> Tuple[ast.AST, ast.AST, ast.AST]:
    # Parse the original function
    original_tree = ast.parse(inspect.getsource(original_func))
    arg_name = original_tree.body[0].args.args[0].arg
    
    # Rename the original function
    original_tree.body[0].name = 'temporary'
    
    # Apply the AST transformation
    replacer = SubscriptReplacer(arg_name)
    original_tree = replacer.visit(original_tree)
    ast.fix_missing_locations(original_tree)

    # Replace dictionary accesses with callmap in the original function
    # This would be similar to the code in SubscriptReplacer

    # Create a new function that applies 'temporary' over an array
    loop_base_func_str = f"""
def {original_func.__qualname__}_loop(Z):
    n = Z.shape[0]
    res = np.zeros((n, 1))
    for i in nb.prange(n):
        res[i, 0] = temporary(Z[i, :])
    return res
    """
    vectorized_base_func_str = f"""
def {original_func.__qualname__}_vectorized(Z):
    return temporary(Z.T)
    """
    loop_func_tree = ast.parse(loop_base_func_str)
    vectorize_func_tree = ast.parse(vectorized_base_func_str)

    return original_tree, loop_func_tree, vectorize_func_tree

def numba_decorate(func_tree: ast.AST, nopython: bool = True, nogil: bool = True, parallel: bool = True) -> ast.AST:
    # # Add Numba JIT decorator
    nb_compyled_func_tree = copy.deepcopy(ast.fix_missing_locations(func_tree))
    numba_decorator = ast.Call(
        func=ast.Attribute(value=ast.Name(id='nb', ctx=ast.Load()), attr='jit', ctx=ast.Load()),
        args=[],
        keywords=[
            ast.keyword(arg='nopython', value=ast.Constant(value=nopython)),
            ast.keyword(arg='nogil', value=ast.Constant(value=nogil)),
            ast.keyword(arg='parallel', value=ast.Constant(value=parallel))
        ]
    )
    nb_compyled_func_tree.body[0].decorator_list.append(numba_decorator)
    nb_compyled_func_tree.body[0].name += '_nb_compyled'
    return ast.fix_missing_locations(nb_compyled_func_tree)

def encapulate(wrap_tree: ast.AST, callmap_tree: ast.AST, original_tree: ast.AST) -> ast.AST:
    wrap_tree.body[0].body.insert(0, callmap_tree.body[0])
    wrap_tree.body[0].body.insert(1, original_tree.body[0])
    return ast.fix_missing_locations(wrap_tree)

def compile_tree(built_func_tree: ast.AST, exec_globals: Dict[str, Any], qualname: str, build_qualifier: str) -> Dict:
    try:
        exec(compile(built_func_tree, filename="<ast>", mode="exec"), exec_globals)
        return {build_qualifier: exec_globals[qualname + build_qualifier]}
    except Exception as e:
        logger.warning(e)
    return {}

def _prepare_funcs(original_func: ast.AST, mapping: Dict[str, int]) -> Dict[str, Callable]:
    exec_globals = globals().copy()
    exec_globals.update({'np': np, 'nb': nb})
    callmap_func_ast = create_callmap_function_ast(mapping)
    callmap_func_tree = ast.fix_missing_locations(ast.Module(body=[callmap_func_ast], type_ignores=[]))
    original_tree, loop_func_tree, vectorize_func_tree = create_transformed_function_ast(original_func, mapping)

    loop_func_tree = encapulate(loop_func_tree, callmap_func_tree, original_tree)
    vectorize_func_tree = encapulate(vectorize_func_tree, callmap_func_tree, original_tree)
    
    available_funcs = {}
    available_funcs.update(compile_tree(vectorize_func_tree, exec_globals, original_func.__qualname__, '_vectorized'))
    available_funcs.update(compile_tree(loop_func_tree, exec_globals, original_func.__qualname__, '_loop'))
    
    nb_compyled_loop_func_tree = numba_decorate(loop_func_tree)
    nb_compyled_vectorize_func_tree = numba_decorate(vectorize_func_tree)

    available_funcs.update(compile_tree(nb_compyled_vectorize_func_tree, exec_globals, original_func.__qualname__, '_vectorized_nb_compyled'))
    available_funcs.update(compile_tree(nb_compyled_loop_func_tree, exec_globals, original_func.__qualname__, '_loop_nb_compyled'))

    return available_funcs

def make_class_decorator(function_decorator: Callable) -> Callable:
    """
    Creates a class decorator from a given function decorator.

    Args:
        function_decorator (Callable): A function decorator to be applied to class methods.

    Returns:
        Callable: A class decorator.
    """
    @functools.wraps(function_decorator)
    def class_decorator(cls: Type) -> Type:
        """
        The class decorator generated from the function decorator.

        Args:
            cls (Type): The class to which the decorator is applied.

        Returns:
            Type: The decorated class.
        """
        for attr_name, attr_value in cls.__bases__[0].__dict__.items():
            if callable(attr_value) and not attr_name.startswith('_') and attr_name not in cls.__dict__:
                setattr(cls, attr_name, function_decorator(attr_value))
        for attr_name, attr_value in cls.__dict__.items():
             if callable(attr_value) and not attr_name.startswith('_'):
                setattr(cls, attr_name, function_decorator(attr_value))
        return cls
    return class_decorator

def autowrap_pandas_return(fn: Callable) -> Callable:
    """
    Decorator to add validation and error handling to class methods.

    Args:
        fn (Callable): The original method of the class.

    Returns:
        Callable: The decorated method with added validation and error handling.
    """
    @functools.wraps(fn)
    def wrapper(self, *args, **kwargs):
        if self._outside_call:
            self._outside_call = False
            res = fn(self, *args, **kwargs)
            if isinstance(res, pd.DataFrame):
                res = pandopt(res)
            self._outside_call = True
            return res
        return fn(self, *args, **kwargs)
    return wrapper


In [429]:

@make_class_decorator(autowrap_pandas_return)
class pandopt(pd.DataFrame):
    _compiled_func = None
    _outside_call = True

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._compiled_func = {}

    @property
    def __name__(self):
        return functools.reduce(lambda x, y: x + y, self.name_to_index.keys())

    @property
    def colname_to_colnum(self):
        return {k: i for i, k in enumerate(self.columns)}

    @property
    def rowname_to_rownum(self):
        return {k: i for i, k in enumerate(self.index)}
    
    def _compiled_qualifier(self, func_qualifier, mapper):
        return hash(functools.reduce(lambda x, y: f'{x}&{y}', mapper) + func_qualifier)

    def apply(self, func, axis = 0, *args, pandas_fallback = False, **kwargs):
        if pandas_fallback: 
            logger.warning(f'{__class__} finish in pandas fallback for func {func}')
            return super().apply(func, axis = 0, *args, **kwargs)
        if args or kwargs:
            logger.warning(f'{__class__} apply only supports func and axis arguments, using default pandas apply')
            return super().apply(func, axis = 0, *args, **kwargs)
        return pandopt((self._compiled_func.get((name:=self._compiled_qualifier(func_qualifier = func.__qualname__, mapper=(mapper:=self.colname_to_colnum if axis else self.rowname_to_rownum)))) or self._build_apply_versions(func, mapper, name))(self.to_numpy() if axis else self.to_numpy().T), index = self.index if axis else self.columns)

    def _with_fallback_wrap(self, apply_func_dict):
        def _with_protects(*args, **kwargs):
            for key in ('_vectorized_nb_compyled', '_loop_compyled', '_vectorized', '_loop'):
                if key not in apply_func_dict:
                    continue
                try:
                    return apply_func_dict[key](*args, **kwargs)
                except:
                    apply_func_dict.pop(key)
            return self.apply(*args, pandas_fallback = True, **kwargs)
        return _with_protects
    
    def _build_apply_versions(self, func, map, name):
        self._compiled_func[name] = self._with_fallback_wrap(_prepare_funcs(func, map))
        return self._compiled_func[name]

    def rolling(self, window, *args, **kwargs):
        return pandoptRoll(self, window=window, *args, **kwargs)

    
    def groupby(self, *args, **kwargs):
        return pandoptGroupBy(self, *args, **kwargs)

    def resample(self, *args, **kwargs):
        return pandoptResampler(self, *args, **kwargs)

    def expanding(self, *args, **kwargs):
        return pandoptExpanding(self, *args, **kwargs)


    def _apply_with_compiled_func(self, func, data, axis=0, *args, **kwargs):
        name = self._compiled_qualifier(func_qualifier=(func_name:=func.__qualname__), mapper=(mapper:=self.colname_to_colnum if axis else self.rowname_to_rownum))
        compiled_func = self._compiled_func.get(name) or self._build_apply_versions(func, mapper, name)
        return res.T if hasattr((res:=compiled_func(data, *args, **kwargs)), 'T') else res

@make_class_decorator(autowrap_pandas_return)
class pandoptRoll(pd.core.window.rolling.Rolling):
    _outside_call = True
    def __init__(self, df, *args, **kwargs):
        self._idf = df
        super().__init__(df, *args, **kwargs)

    def agg(self, func, *args, **kwargs):
        if isinstance(func, dict):
            # For dictionary input, apply each function to the respective column
            result = {}
            for column, func_name in func.items():
                if func_name in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median']:
                    result[column] = super().agg({column: func_name})
                else:
                    result[column] = self.apply(lambda x: x[column].pipe(func_name), *args, **kwargs)
            return pandopt(result)
        elif func in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median']:
            return super().agg(func, *args, **kwargs)
        else:
            return self.apply(func, *args, **kwargs)
        
    def apply(self, func, *args, pandas_fallback=False, **kwargs):
        try:
            return pandopt(self._idf._apply_with_compiled_func(func, sliding_window_view(df.to_numpy(), self.window, axis=0), axis=2, *args, **kwargs), columns=df.columns, index=df.index[self.window-1:])
        except Exception as e:
            logger.warning(f'Error in pandoptRoll apply: {e}')
            if pandas_fallback:
                return super().apply(func, *args, **kwargs)
            raise
            
@make_class_decorator(autowrap_pandas_return)
class pandoptGroupBy(pd.core.groupby.DataFrameGroupBy):
    _outside_call = True
    def __init__(self, df, *args, **kwargs):
        self._idf = df
        super().__init__(df, *args, **kwargs)

    def agg(self, func, *args, **kwargs):
        if isinstance(func, dict):
            # For dictionary input, apply each function to the respective column
            result = {}
            for column, func_name in func.items():
                if func_name in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median']:
                    result[column] = super().agg({column: func_name})
                else:
                    result[column] = self.apply(lambda x: x[column].pipe(func_name), *args, **kwargs)
            return pandopt(result)
        elif func in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median']:
            return super().agg(func, *args, **kwargs)
        else:
            return self.apply(func, *args, **kwargs)

    def apply(self, func, *args, pandas_fallback=False, **kwargs):
        try:
            results = []
            for name, group in self:
                result_array = self._idf._apply_with_compiled_func(func, group.to_numpy(), axis=1, *args, **kwargs)
                
                # If the function returns a single value per group
                if result_array.ndim == 1 or result_array.shape[1] == 1:
                    result_df = pandopt(result_array, index=group.index, columns=[func.__name__])
                else:
                    result_df = pandopt(result_array, index=group.index, columns=group.columns)
                
                results.append(result_df)

            return pd.concat(results, axis=0)
        except Exception as e:
            logger.warning(f'Error in pandoptGroupBy apply: {e}')
            if pandas_fallback:
                return super().apply(func, *args, **kwargs)
            raise

@make_class_decorator(autowrap_pandas_return)
class pandoptResampler(pd.core.resample.Resampler):
    _outside_call = True
    def __init__(self, df, *args, **kwargs):
        self._idf = df
        super().__init__(df, *args, **kwargs)

    def agg(self, func, *args, **kwargs):
        if isinstance(func, dict):
            results = {col: (super().agg({col: f}) if f in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median'] else self.apply(lambda x: x[col].pipe(f), *args, **kwargs)) for col, f in func.items()}
            return pandopt(results)
        elif func in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median']:
            return super().agg(func, *args, **kwargs)
        else:
            return self.apply(func, *args, **kwargs)

    def apply(self, func, *args, pandas_fallback=False, **kwargs):
        try:
            if pandas_fallback:
                return super().apply(func, *args, **kwargs)
            return self._idf._apply_with_compiled_func(func, self._idf.to_numpy(), *args, **kwargs)
        except Exception as e:
            logger.warning(f'Error in pandoptResampler apply: {e}')
            return super().apply(func, *args, **kwargs)

@make_class_decorator(autowrap_pandas_return)
class pandoptExpanding(pd.core.window.expanding.Expanding):
    _outside_call = True
    def __init__(self, df, *args, **kwargs):
        self._idf = df
        super().__init__(df, *args, **kwargs)

    def agg(self, func, *args, **kwargs):
        if isinstance(func, dict):
            results = {col: (super().agg({col: f}) if f in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median'] else self.apply(lambda x: x[col].pipe(f), *args, **kwargs)) for col, f in func.items()}
            return pandopt(results)
        elif func in ['mean', 'std', 'sum', 'min', 'max', 'count', 'median']:
            return super().agg(func, *args, **kwargs)
        else:
            return self.apply(func, *args, **kwargs)

    def apply(self, func, *args, pandas_fallback=False, **kwargs):
        try:
            if pandas_fallback:
                return super().apply(func, *args, **kwargs)
            return self._idf._apply_with_compiled_func(func, self._idf.to_numpy(), *args, **kwargs)
        except Exception as e:
            logger.warning(f'Error in pandoptExpanding apply: {e}')
            return super().apply(func, *args, **kwargs)



In [427]:
df=pd.DataFrame(data=np.random.randn(100000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
df['E'] = df['A'] // df['C']
dfx = pandopt(df)

%time df.groupby('E').agg(Measssn)
%time dfx.groupby('E').agg(Measssn)


CPU times: user 103 ms, sys: 300 µs, total: 103 ms
Wall time: 103 ms


AttributeError: 'pandoptGroupBy' object has no attribute '_outside_call'

In [423]:
def Measssn(X):
    return np.mean(X, axis = 0)

def neow(X):
    return np.mean(X)

df=pd.DataFrame(data=np.random.randn(1000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)
%time dfx.rolling(10).apply(Measssn)
# dfx._compiled_func
# dfx.rolling(10).mean()
dfx['E'] = dfx['A'] // dfx['C']
dfx.groupby('E').apply(neow), dfx.rolling(10).apply(Measssn)

CPU times: user 21.4 ms, sys: 3.68 ms, total: 25.1 ms
Wall time: 24.5 ms


Error in pandoptGroupBy apply: 'float' object has no attribute 'ndim'


AttributeError: 'float' object has no attribute 'ndim'

In [396]:
sliding_window_view(df.to_numpy().T, 3, axis=0).shape

(2, 1000, 3)

In [425]:
def MIMMEAN(x):
    return np.std(x, axis = 0)

df=pd.DataFrame(data=np.random.randn(100000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)



#%time a = df.rolling(10).apply(f)
%time b = dfx.rolling(10).apply(MIMMEAN)
%time c = dfx.rolling(10).std()



a , b, c

CPU times: user 4.86 s, sys: 1.42 s, total: 6.28 s
Wall time: 6.29 s
CPU times: user 5.97 s, sys: 1.74 s, total: 7.71 s
Wall time: 7.71 s


(            A         B         C         D
 0         NaN       NaN       NaN       NaN
 1         NaN       NaN       NaN       NaN
 2         NaN       NaN       NaN       NaN
 3         NaN       NaN       NaN       NaN
 4         NaN       NaN       NaN       NaN
 ..        ...       ...       ...       ...
 995  0.448690  0.352566  0.305297 -0.704883
 996  0.478569  0.402863  0.284317 -0.701448
 997  0.010246  0.422342 -0.040771 -0.643033
 998 -0.128343  0.393618 -0.144073 -0.658981
 999 -0.155979  0.356666 -0.197278 -0.856816
 
 [1000 rows x 4 columns],
                  A         B         C         D
 9         1.025317  0.908619  1.024565  1.167187
 10        1.018969  0.893918  1.024359  1.165360
 11        0.970301  0.798655  1.109088  0.993432
 12        1.016979  0.790451  1.147673  0.783343
 13        1.003597  0.755846  1.186392  0.839486
 ...            ...       ...       ...       ...
 99999995  0.805636  0.758432  0.798632  0.759964
 99999996  0.848169  0.770966  0

In [196]:
df=pd.DataFrame(data=np.random.randn(1000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)
df.rolling(10

Rolling [window=10,center=False,axis=0,method=single]

In [178]:
def f(x):
    return np.mean(x)

df=pd.DataFrame(data=np.random.randn(1000000,4), columns = ['A', 'B', 'C', 'D']).astype(np.float32)
dfx = pandopt(df)
dfx.rolling(10)

%time df.rolling(10).mean()
%time dfx.rolling(10)



Error in pandoptRoll apply: 'RangeIndex' object has no attribute 'columns'


CPU times: user 45.2 ms, sys: 57 µs, total: 45.3 ms
Wall time: 45.2 ms


KeyboardInterrupt: 

In [154]:
isinstance(df.rolling(10, center=True), pd.core.window.rolling.Rolling)

True